In [63]:
import pandas as pd
import re
import csv
import time
import numpy as np
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

//anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
input_dir = 'paymo_input/'
batch_file = 'batch_payment.csv'
stream_file = 'stream_payment.csv'
batch_path = input_dir + batch_file
stream_path = input_dir + stream_file

test_file = 'batch_test_2.csv'
test_path = input_dir + test_file

In [4]:
batch_dict = {}
batch_dict['time'] = {}
batch_dict['id1'] = {}
batch_dict['id2'] = {}
batch_dict['amount'] = {}
batch_dict['message'] = {}

with open(batch_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        batch_dict['time'][i] = row['time']
        batch_dict['id1'][i] = row[' id1']
        batch_dict['id2'][i] = row[' id2']
        batch_dict['amount'][i] = row[' amount']
        batch_dict['message'][i] = row[' message']
        
df_batch = pd.DataFrame.from_dict(batch_dict)

In [5]:
stream_dict = {}
stream_dict['time'] = {}
stream_dict['id1'] = {}
stream_dict['id2'] = {}
stream_dict['amount'] = {}
stream_dict['message'] = {}

with open(stream_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        stream_dict['time'][i] = row['time']
        stream_dict['id1'][i] = row[' id1']
        stream_dict['id2'][i] = row[' id2']
        stream_dict['amount'][i] = row[' amount']
        stream_dict['message'][i] = row[' message']
        
df_stream = pd.DataFrame.from_dict(stream_dict)

In [6]:
## BIG PROBLEM - SKIPPED OVER A BUNCH OF ENTRIES
batch_dict['message'][377592]
#print(len(batch_dict['message'].keys()))

' 🇨🇴🇨🇴🇨🇴🇨🇴👍🏼🎉 '

In [7]:
#dictionary columns are randomly ordered - reorder as expected
df_batch = df_batch[['time','id1','id2','amount','message']]
df_stream = df_stream[['time','id1','id2','amount','message']]

In [8]:
df_batch.tail()

,time,id1,id2,amount,message
3934578,2016-11-02 09:49:29,20942,826,39.40,🍻🍾🍷
3934579,2016-11-02 09:49:29,2464,2425,6.03,Taco tuesday
3934580,2016-11-02 09:49:29,68862,25922,22.48,Comcast
3934581,2016-11-02 09:49:29,66910,21661,21.28,Signs
3934582,2016-11-02 09:49:29,64593,18179,31.57,blue kiwi


In [9]:
givers = df_batch.groupby('id1')
receivers = df_batch.groupby('id2')
partners_1 = {}
partners_2 = {}

In [93]:
## find all transactions where user <user_id> was giver, then find list of partners in those transactions
for user_id,transactions in givers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try:
        partners_1[int(user_id)] = list(givers.get_group(user_id)['id2'].astype(int))
   
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)
    
## same as before for all transactions where <user_id> was receiver
for user_id,transactions in receivers:
    
    #store list of all transaction partners as list (easiest type to extend later)
    try: 
        partners_2[int(user_id)] = list(receivers.get_group(user_id)['id1'].astype(int))
        
    #some lines of batch_payment.txt and stream_payment.txt are off - omit malformed entries
    except (KeyError, ValueError) as BadLine:
        print("Skipping invalid key:",user_id)

Skipping invalid key:  no. Even if the union were a matter of economic indifference
Skipping invalid key:  and even if it were to be disadvantageous from the economic standpoint
time: 1min 33s


In [98]:
## it's possible that some users only show up as givers and others only as receivers - combine to master list of all IDs
## in actuality for the provided batch_payment.txt all users show up as givers at least once, but not safe to assume
user_list_1 = np.array(list(partners_1.keys()))
user_list_2 = np.array(list(partners_2.keys()))
user_list = np.unique(np.concatenate([user_list_1,user_list_2]))

(77360,)
(77336,)
(77360,)
time: 24.1 ms


In [106]:
partners = {}

#cycle through all users and agglomerate partners from all transactions
for user_id in user_list:
    
    pp = []
    
    if user_id in partners_1.keys():
        pp += partners_1[user_id]
        
    if user_id in partners_2.keys():
        pp += partners_2[user_id]
        
    #reduce to (sorted) list of all unique partners
    partners[user_id] = np.unique(pp)

time: 1.78 s


In [113]:
len(partners.keys())

77360

time: 1.86 ms


In [79]:
if 10000 in partners_1.keys():
    print(1)

1
time: 949 µs


In [69]:
    #merge into single list, use .unique() to weed out duplicates
    #stations = pd.concat([start_stations, end_stations], axis=0, ignore_index = True)
    #unique_stations = stations.unique()
    #visited_stations[k] = len(unique_stations)
    #partners[user] = 

time: 1.08 ms


In [ ]:
users = df.groupby('id1')

#store the number of visited stations in dictionary format, using each bikeid as key
visited_stations = {}

for k,v in bikes:
    #following gets list of start and end stations visited by each bike
    start_stations = bikes.get_group(k)['start station id']
    end_stations = bikes.get_group(k)['end station id']
    
    #merge into single list, use .unique() to weed out duplicates
    stations = pd.concat([start_stations, end_stations], axis=0, ignore_index = True)
    unique_stations = stations.unique()
    visited_stations[k] = len(unique_stations)